# Master course in Object Recognition
## Practice 1

### Title: Deep learning advanced architectures

The goal is to practice advanced deep learning architectures for multi-label classification in [Pascal VOC dataset](http://host.robots.ox.ac.uk/pascal/VOC/voc2007/index.html). We specifically check ResNet50, Inception and MobileNet. We will see 1) how pretrained ResNet50 on imagenet performs on multi-label images, 2) how to modify classification head and 3) implementation of F1 metric.

### NOTES

- Hyperparameters are modifiable,
- The dataset is PASCAL VOC 2012,
- The code uses the KERAS library,
- The code can run in google colab.
- How to finetune on a pretrained model not included (i.e. freeze the pretrained network and train the head, then finetune everything),
- No validation set has been defined. The test and validation sets are the same.

In [1]:
# Imports

from tensorflow.keras import optimizers
from tensorflow.keras.models import Model
from keras.layers import (
    Dense,
    GlobalAveragePooling2D,
)
import tensorflow.keras.applications as app
import pandas as pd
import csv
import time

In [2]:
# Importing from .py files

from metrics import f1_metric, mean_average_precision, subset_accuracy_metric
from config import *
from experiment_config import experiments
from train_and_test import train_and_test
from load_data import load_data, create_dataset

In [ ]:
# Run model experiments
exp_name = "model-experiments"
for exp in experiments[exp_name]:
    print(f"Defining model: {exp.title}")

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    train_and_test(
        model, base_model, exp_name, exp, train_list, test_list
    )

Defining model: resnet50 pretraining warmup
Starting training: resnet50 pretraining warmup
Freezing base model layers...
Time taken for training one epoch: 15.08s
Epoch 0 - Train Loss: 0.6833, Acc: 0.4212, F1: 0.0783, mAP: 0.1362
Time taken for testing one epoch: 3.94s
Epoch 0 - Test Loss: 0.4523, Acc: 0.5612, F1: 0.0729, mAP: 0.2657
Time taken for training one epoch: 12.15s
Epoch 1 - Train Loss: 0.3921, Acc: 0.6542, F1: 0.1450, mAP: 0.4163
Time taken for testing one epoch: 3.94s
Epoch 1 - Test Loss: 0.3195, Acc: 0.6032, F1: 0.2188, mAP: 0.3918
Time taken for training one epoch: 20.75s
Epoch 2 - Train Loss: 0.4609, Acc: 0.5825, F1: 0.1921, mAP: 0.3961
Time taken for testing one epoch: 3.89s
Epoch 2 - Test Loss: 0.3062, Acc: 0.6022, F1: 0.0000, mAP: 0.3994
Results saved to /Users/sheena/Development/master-dev/semester-2/mai-object-recognition/practicals/p1/data/02_results/model-experiments.csv
Model saved to /Users/sheena/Development/master-dev/semester-2/mai-object-recognition/practica

In [ ]:
# Determine the best experiment of the 9 model experiments

df = pd.read_csv(RESULTS_DIR / f"model-experiments.csv")
best_id = df.loc[df["Test mAP"].idxmax(), "ID"]

best_model_experiment_config = next(
    exp for exp in experiments["model-experiments"] if exp.id == best_id
)

best_model_experiment_config

In [ ]:
# Run hyperparameter experiments

exp_name = "hyperparameter-experiments"
for exp in experiments[exp_name]:
    print(f"Defining model: {exp.title}")
    print(
        f"Reusing net_name: {best_model_experiment_config.net_name}, train_from_scratch: {best_model_experiment_config.train_from_scratch}, warm_up: {best_model_experiment_config.warm_up} from best model experiment"
    )
    exp.net_name = best_model_experiment_config.net_name
    exp.train_from_scratch = best_model_experiment_config.train_from_scratch
    exp.warm_up = best_model_experiment_config.warm_up

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, base_model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

In [ ]:
# Determine the best experiment of the 9 hyperparameter experiments

df = pd.read_csv(RESULTS_DIR / f"hyperparameter-experiments.csv")
best_id = df.loc[df["Test mAP"].idxmax(), "ID"]

best_hyperparameter_experiment_config = next(
    exp for exp in experiments["hyperparameter-experiments"] if exp.id == best_id
)

best_hyperparameter_experiment_config

In [ ]:
# Run augmentation experiments

exp_name = "augmentation-experiments"
for exp in experiments[exp_name]:
    print(f"Defining model: {exp.title}")
    print(
        f"Reusing net_name: {best_hyperparameter_experiment_config.net_name}, train_from_scratch: {best_hyperparameter_experiment_config.train_from_scratch}, warm_up: {best_hyperparameter_experiment_config.warm_up} from best hyperparameter experiment"
    )
    exp.net_name = best_hyperparameter_experiment_config.net_name
    exp.train_from_scratch = best_hyperparameter_experiment_config.train_from_scratch
    exp.warm_up = best_hyperparameter_experiment_config.warm_up
    exp.batch_size = best_hyperparameter_experiment_config.batch_size
    exp.learning_rate = best_hyperparameter_experiment_config.learning_rate
    exp.loss = best_hyperparameter_experiment_config.loss
    exp.last_layer_activation = (
        best_hyperparameter_experiment_config.last_layer_activation
    )

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, base_model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )

In [ ]:
# Run classifier head experiments

exp_name = "classfier_head_experiments"
for exp in experiments[exp_name]:
    print(f"Defining model: {exp.title}")
    print(
        f"Reusing net_name: {best_hyperparameter_experiment_config.net_name}, train_from_scratch: {best_hyperparameter_experiment_config.train_from_scratch}, warm_up: {best_hyperparameter_experiment_config.warm_up} from best hyperparameter experiment"
    )
    exp.net_name = best_hyperparameter_experiment_config.net_name
    exp.train_from_scratch = best_hyperparameter_experiment_config.train_from_scratch
    exp.warm_up = best_hyperparameter_experiment_config.warm_up
    exp.batch_size = best_hyperparameter_experiment_config.batch_size
    exp.learning_rate = best_hyperparameter_experiment_config.learning_rate
    exp.loss = best_hyperparameter_experiment_config.loss
    exp.last_layer_activation = (
        best_hyperparameter_experiment_config.last_layer_activation
    )

    # Select the corresponding network class
    mynet = getattr(getattr(app, exp.net_name[0]), exp.net_name[1])

    # Create the base pre-trained model
    base_model = (
        mynet(include_top=False)
        if exp.train_from_scratch
        else mynet(weights="imagenet", include_top=False)
    )

    # Add a global spatial average pooling layer
    x = base_model.output
    x = GlobalAveragePooling2D()(x)
    x = Dense(1024, activation="relu")(x)  # Fully connected layer
    predictions = Dense(num_classes, activation=exp.last_layer_activation)(x)

    # Create the model
    model = Model(inputs=base_model.input, outputs=predictions)

    train_dataset = train_datasets[exp.batch_size]
    test_dataset = test_datasets[exp.batch_size]

    train_and_test(
        model, base_model, exp_name, exp, train_dataset, test_dataset, train_list, test_list
    )